# Importing Libraries

In [263]:
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Property URLs extraction from multiple city URLs

In [379]:
list_city_urls = []
lst = []
file = open("City_URLs.txt","r")
for line in file:
    lst.append(line)

for i in lst:
    x = i.rstrip("\n")
    list_city_urls.append(x)
len(list_city_urls)


23

In [380]:
all_urls = []
for x in list_city_urls:
    req = Request(x, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BS(webpage,'html.parser')
    listcontent = soup.findAll('a',attrs={'class':"list-card-link",'tabindex':"0"},href =True)
    for links in listcontent:
        all_urls.append(links['href'])
df_url = pd.DataFrame({'Urls':all_urls})
df_url.head()

,Urls
0,https://www.zillow.com/homedetails/2200-Geller...
1,https://www.zillow.com/homedetails/116-Mountai...
2,https://www.zillow.com/homedetails/617-Joaquin...
3,https://www.zillow.com/homedetails/224-Village...
4,https://www.zillow.com/homedetails/376-Avalon-...


## Writing the URLs to a csv file 

In [382]:
df_url.to_csv('all_urls.csv',index=False) 

# Reading the csv file for the URLs

In [383]:
urls = pd.read_csv('all_urls.csv')
urls.head()

,Urls
0,https://www.zillow.com/homedetails/2200-Geller...
1,https://www.zillow.com/homedetails/116-Mountai...
2,https://www.zillow.com/homedetails/617-Joaquin...
3,https://www.zillow.com/homedetails/224-Village...
4,https://www.zillow.com/homedetails/376-Avalon-...


In [149]:
P_Price = []
P_Year = []
P_Apt_type = []
P_Heating = []
P_Cooling = []
P_Parking = []
P_Zestimate = []
P_Address =[]
P_Beds = []
P_Baths = []
P_Area = []
P_HOA = []

for x in urls['Urls'][850:885]:
    req = Request(str(x), headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    webstring = str(webpage)
    soup = BS(webpage,'html.parser')
    
    homeprice = []
    price = soup.findAll('span',attrs={'class':'ds-value'})
    for tag in price:
        homeprice.append(tag.text)
    if len(homeprice)==0:
        P_Price.append(0)
    else:
        P_Price.append(homeprice[0])
        
    homefacts = []
    homefactvalue = soup.findAll('span',attrs={'class':"ds-body ds-home-fact-value"})

    for tag in homefactvalue:
        homefacts.append(tag.contents[0])
    if len(homefacts)==0:
        P_Apt_type.append('0')
        P_Year.append('0')
        P_Heating.append('0')
        P_Cooling.append('0')
        P_Parking.append('0')
    else:
        P_Apt_type.append(homefacts[0])
        P_Year.append(homefacts[1])
        P_Heating.append(homefacts[2])
        P_Cooling.append(homefacts[3])
        P_Parking.append(homefacts[4])
    index = soup.findAll('div',attrs={'class':'Spacer-sc-17suqs2-0 gsRmiW'})
    if len(index)==0:
        P_Zestimate.append('0')
    else:
        P_Zestimate.append(index[0].text)
    
    
    bedbath = []       
    
    index = soup.findAll('span',attrs={'class':"ds-bed-bath-living-area"})
    for tag in index:
        bedbath.append(tag.contents[0].text)
    if len(bedbath)>=3:
        P_Beds.append(bedbath[0])
        P_Baths.append(bedbath[1])
        P_Area.append(bedbath[2])
    elif len(bedbath)==0:
        P_Beds.append('0')
        P_Baths.append('0')
        P_Area.append('0')
    elif len(bedbath)==1:
        P_Beds.append(bedbath[0])
        P_Baths.append(bedbath[1])
        P_Area.append('0')
    else:
        P_Beds.append(bedbath[0])
        P_Baths.append('0')
        P_Area.append('0')
        
        

    address = soup.findAll('h1',attrs={'class':"ds-address-container"})
    addr1 = []
    addr2 = []
    for tag in address:
        addr1.append(tag.contents[0].text)
        addr2.append(tag.contents[1].text)
    if (len(addr1) | len(addr2)) == 0:
        P_Address.append('0')
    else:
        P_Address.append(addr1[0]+' '+addr2[0])

print(len(P_Address),len(P_Beds),len(P_Baths),len(P_Area))

print(len(P_Price),len(P_Year),len(P_Apt_type),len(P_Heating),len(P_Cooling),len(P_Parking))


35 35 35 35
35 35 35 35 35 35


In [155]:
data18 = pd.DataFrame({"Price":P_Price,"Built_Year":P_Year,"Apt_Type":P_Apt_type,"Heating":P_Heating,"Cooling":P_Cooling,
                     "Parking":P_Parking,'Address':P_Address,'Bedrooms':P_Beds,'Bath_Spaces':P_Baths,'Area':P_Area,
                     'Zestimate':P_Zestimate},index = [x for x in range(850,885)])


,Price,Built_Year,Apt_Type,Heating,Cooling,Parking,Address,Bedrooms,Bath_Spaces,Area,Zestimate
850,"$1,349,000",2011,Townhouse,Forced air,Central,2 spaces,"428 San Eduardo Ter, Sunnyvale, CA 94085",3,4,"1,516","$1,390,224"
851,"$1,299,000",1969,Single Family,Forced air,Central,2 spaces,"1052 E Evelyn Ave, Sunnyvale, CA 94086",3,2,"1,198","$1,339,877"
852,"$1,998,000",1958,Single Family,Radiant,None,2 spaces,"1607 Edmonton Ave, Sunnyvale, CA 94087",4,2,"1,890","$2,298,087"
853,"$1,148,000",1986,Townhouse,Forced air,None,2 spaces,"979 Belmont Ter UNIT 4, Sunnyvale, CA 94086",2,3,"1,329","$1,225,007"
854,"$1,899,000",1948,Single Family,Forced air,No Data,10 spaces,"1350 Ramon Dr, Sunnyvale, CA 94087",3,3,"2,491","$2,193,388"


### As there is a timeout issue, ie a request being called from a single source multiple times leads to a timeout issue, we have collected the data in clusters of 50 and appended them later.

In [151]:
data = data.append(data18)

In [255]:
data.to_csv('Real_data.csv', header = True, index = False)

In [333]:
df = pd.read_csv("RealEstateData.csv")

In [334]:
df.head()

,Price,Built_Year,Apt_Type,Heating,Cooling,Parking,Address,Bedrooms,Bath_Spaces,Area,Zestimate,zip_code
0,1195000,1989,Single Family,Forced air,None,2 spaces,"116 Mountain Rd, South San Francisco, CA 94080",3,3.0,"1,580",1304120,94080
1,1188000,1955,Single Family,Forced air,None,Attached Garage,"617 Joaquin Dr, South San Francisco, CA 94080",3,2.0,"1,540",1264270,94080
2,928000,1944,Single Family,Forced air,None,Attached Garage,"224 Village Way, South San Francisco, CA 94080",3,2.0,"1,699",1034580,94080
3,1095000,1955,Single Family,Forced air,None,Attached Garage,"376 Avalon Dr, South San Francisco, CA 94080",4,2.0,"1,650",1237270,94080
4,888000,1944,Single Family,Forced air,None,1 space,"742 W Orange Ave, South San Francisco, CA 94080",2,1.0,"1,000",968716,94080


## Data Cleaning

In [335]:
price = []
for i in df['Price']:
    x = str(i)
    res = ''.join([i for i in x if i.isdigit()])
    price.append(res)
price = list(map(int,price))
df['Price'] = price

In [336]:
area = []
for i in df['Area']:
    x = str(i)
    res = ''.join([i for i in x if i.isdigit()])
    area.append(res)
area = list(map(int,area))
df['Area'] = area

In [337]:
year = []
for i in df['Built_Year']:
    if i == 'No Data':
        year.append(0)
    else:
        year.append(i)
year = list(map(int,year))
df['Built_Year'] = year

In [338]:
df['Apt_Type'] = list(map(str,df['Apt_Type']))

In [339]:
zestimate = []
for i in df['Zestimate']:
    x = str(i)
    res = ''.join([i for i in x if i.isdigit()])
    zestimate.append(res)
price = list(map(int,price))
df['Zestimate'] = zestimate


In [341]:
city = []
zip_code = []
address = []

for i in range(len(df)):
    if 'nan' in str(df.loc[i,'Address']):
        zip_code.append(0)
    else:
        zip_code.append(int(df.loc[i,'Address'][-5:]))  

df2 = df.assign(zip_code = zip_code)

In [376]:
df2['Bedrooms'] = df2['Bedrooms'].replace('--',0)
df2['Bath_Spaces'] = df2['Bath_Spaces'].replace('--',0)
df2['Zestimate'] = df2['Zestimate'].replace('',0)

## Converting to Numeric datatype

In [344]:
df2['Price'] = pd.to_numeric(df2['Price'])
df2['Bath_Spaces'] = pd.to_numeric(df2['Bath_Spaces'])
df2['Area'] = pd.to_numeric(df2['Area'])
df2['Zestimate'] = pd.to_numeric(df2['Zestimate'])
df2['zip_code'] = pd.to_numeric(df2['zip_code'])

In [360]:
city = []
for i in range(len(df2['Address'])):
    string = str(df2['Address'][i])
    
    index = re.findall(', \xa0(.*?), [A-Z]{2} \d{5}$',string)
    if len(index) != 0:
        city.append(index[0])
    else:
        city.append('0')
df3 = df2.assign(city = city)

## Removing rows that contain zeroes

In [372]:
df3.drop(df3[df3.Zestimate == 0].index, inplace =True)
df3.drop(df3[df3.Price == 0].index, inplace =True)
df3.drop(df3[df3.Built_Year == 0].index, inplace= True)
df3.drop(df3[df3.Bedrooms == 0].index, inplace= True)
df3.drop(df3[df3.Bath_Spaces == 0].index, inplace= True)
df3.drop(df3[df3.Area == 0].index, inplace= True)

## Checking NAs and Datatypes

In [375]:
df3.isna().sum()

Price          0
Built_Year     0
Apt_Type       0
Heating        0
Cooling        0
Parking        0
Address        0
Bedrooms       0
Bath_Spaces    0
Area           0
Zestimate      0
zip_code       0
city           0
dtype: int64

In [377]:
df3.dtypes

Price            int64
Built_Year       int64
Apt_Type        object
Heating         object
Cooling         object
Parking         object
Address         object
Bedrooms         int64
Bath_Spaces    float64
Area             int64
Zestimate        int64
zip_code         int64
city            object
dtype: object

## Writing the dataset to a csv file

In [373]:
df3.to_csv("Bay_Area_Housing_Data_Final.csv",header = True, index = False)